In [185]:
import requests
import datetime
import pandas as pd
from bs4 import BeautifulSoup as bs

In [167]:
# 收集所有當前的展覽頁面
def collectExpoPage():
    HSExpoPages = []
    i = 1
    while i >0 : 
        url = "https://www.huashan1914.com/w/huashan1914/exhibition?index={}".format(i)
        if len(getHSRawData(url)) > 0:
            HSExpoPages.append(url)
            i += 1
        else:
            break
    return HSExpoPages

In [159]:
# 得到華山展演活動的個分頁的HTML
def getHSRawData(url):
    res = requests. get(url)
    soup = bs(res.text,"lxml")
    ExpoRawPage = soup.select("li.item-static")
    return ExpoRawPage

In [259]:
# 解析每一個expo
def getExpoInfo(expo):

    expoTitle = expo.select("div.card-text-name")[0].text
    expoDate = expo.select("div.event-date")[0].text.replace("\n","") #去掉空白行
    expoDate = proceedHSExpoDate(expoDate)
    #     exhibitionTime = [time.text for time in expo.select("div.event-time span")] #暫不處理時間
    expoLink = expo.select("a")[0]["href"]
    ID = expoLink.replace("/w/huashan1914/exhibition_","")
    expoLink = "https://www.huashan1914.com" + expoLink
    expoCatagory = expo.select("div.event-list-type")[0].text.replace("\n"," ").replace(" ","")

    expoData = {
        "place"        :  "華山",
        "title"        :  expoTitle,
        "catagory"     :  expoCatagory,
        "startDate"    :  expoDate[0],
        "endDate"      :  expoDate[1],
        "id"           :  ID,
        "link"         :  expoLink,  
#       "imagePath" todo
    }
    return expoData

In [255]:
# 處理expo開始以及結束的日期並轉成datetime格式
def proceedHSExpoDate(date):
    date = date.split(" - ") #用空白及dash切割開始以及結束日期       
    startDate = date[0].replace(".","-")
    startDate = datetime.datetime.strptime(startDate,"%Y-%m-%d").date()

    if len(date) == 2 :
        date = date[1].split(".")
        if len(date[0]) != 4 :
            date.insert(0,"2019")
        endDate = "-".join(date)
        endDate = datetime.datetime.strptime(endDate,"%Y-%m-%d").date()
    else:
        endDate = startDate
    
    return [startDate, endDate]

In [224]:
# 存檔
def saveHSExpo(allDataList, fileName):

    df = pd.DataFrame(allDataList)
    df = df[["place","title","catagory", "startDate","endDate", "id","link"]]
    df.to_csv(fileName, index = False)

In [200]:
# 1.找出現有的expo page
currentExpoPage = collectExpoPage()
print("找出所有網頁OK")

找出所有網頁OK


In [228]:
# 2.找出每個展覽li
allExposList = []
for pageUrl in currentExpoPage:
    allExposList += getHSRawData(pageUrl)
print("找出每個展覽li OK")   

找出每個展覽li OK


In [264]:
# 3.取得所有展覽資訊

allData=[]
for expo in allExposList:
    expoData = getExpoInfo(expo)
    allData.append(expoData)
print("取得所有展覽資訊 OK") 



取得所有展覽資訊 OK


In [266]:
# 4.存檔
saveHSExpo(allData, "20191212_HS.csv")
print("存檔 OK")

存檔 OK


In [235]:
df = pd.DataFrame(allData)
df

,place,title,catagory,startDate,endDate,id,link
0,華山,神腦原鄉踏查紀錄片競賽頒獎暨特映會,展演活動,2019-11-08,2020-01-06,19112515465572411,https://www.huashan1914.com/w/huashan1914/exhi...
1,華山,NEWVIEW AWARDS 2019 國際新銳 VR 作品體驗展,展演活動,2019-11-08,2020-01-06,19120114365641313,https://www.huashan1914.com/w/huashan1914/exhi...
2,華山,美業文青潮市集,展演活動市集活動,2019-11-08,2020-01-06,19120321542315303,https://www.huashan1914.com/w/huashan1914/exhi...
3,華山,第三屆青年公益實踐計畫成果發表會,展演活動,2019-11-08,2020-01-06,19120419303352332,https://www.huashan1914.com/w/huashan1914/exhi...
4,華山,熱青年Action！第二屆互動展,論壇講座,2019-11-08,2020-01-06,19102915123130443,https://www.huashan1914.com/w/huashan1914/exhi...
5,華山,2019第七屆 華文朗讀節-故事的力量,展演活動,2019-11-08,2020-01-06,19112315573786882,https://www.huashan1914.com/w/huashan1914/exhi...
6,華山,【華山文旅學 X 相遇地方】地方創生的風土設計課-青年如何參與地方創生，創造自己的志業與事業,論壇講座,2019-11-08,2020-01-06,19112612263626368,https://www.huashan1914.com/w/huashan1914/exhi...
7,華山,2019台灣插畫設計週 《靈感画市》,市集活動,2019-11-08,2020-01-06,19120417311675535,https://www.huashan1914.com/w/huashan1914/exhi...
8,華山,「絕景・只見線——魅惑吳心緹の日本最美鐵道風景」攝影展,展演活動,2019-11-08,2020-01-06,19120521554304203,https://www.huashan1914.com/w/huashan1914/exhi...
9,華山,Garmin「vívomove系列獨一有二快閃店」,展演活動,2019-11-08,2020-01-06,19120916304174407,https://www.huashan1914.com/w/huashan1914/exhi...
